# Kreditberechnung

In [ ]:
#%pip install numpy
%pip install seaborn

Traceback (most recent call last):
  File "/Users/andreasschwab/.vscode/extensions/ms-python.python-2025.4.0-darwin-x64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2
    %pip install seaborn
    ^
SyntaxError: invalid syntax



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from IPython.display import display

# ====== Bank Names and Styles ======
bank_profiles = {
    3.0: {"name": "Basisniveau", "color": "#005baa", "marker": "D"},
    3.21: {"name": "Hypofin", "color": "#e37222", "marker": "s"}, 
    3.57: {"name": "BBank", "color": "#a5c261", "marker": "^"},
    3.71: {"name": "Coba", "color": "#c4071b", "marker": "v"},
    4.0: {"name": "Zu teuer", "color": "#6d3f96", "marker": "p"}
}

# ====== Loan Calculation Function ======
def calculate_loan_fixed_monthly(principal, monthly_payment, annual_rate):
    monthly_rate = annual_rate / 12
    remaining = principal
    total_interest = 0
    yearly_data = []
    year_counter = 0
    yearly_interest = 0
    yearly_repayment = 0
    
    while remaining > 0:
        interest = remaining * monthly_rate
        repayment = min(monthly_payment - interest, remaining)
        remaining -= repayment
        total_interest += interest
        
        yearly_interest += interest
        yearly_repayment += repayment
        year_counter += 1
        
        if year_counter == 12:
            yearly_data.append({
                'Year': len(yearly_data) + 1,
                'Interest': round(yearly_interest),
                'Repayment': round(yearly_repayment),
                'Remaining': round(remaining),
                'Rate': annual_rate
            })
            yearly_interest = 0
            yearly_repayment = 0
            year_counter = 0
    
    if year_counter > 0:
        yearly_data.append({
            'Year': len(yearly_data) + 1,
            'Interest': round(yearly_interest),
            'Repayment': round(yearly_repayment),
            'Remaining': round(remaining),
            'Rate': annual_rate
        })
    
    return pd.DataFrame(yearly_data), total_interest, len(yearly_data)

# ====== Parameters ======
loan_amount = 350000
fixed_monthly_payment = 1500  # € per month
interest_rates = sorted(bank_profiles.keys())

# ====== Calculations ======
results = []
all_data = pd.DataFrame()
max_years = 0

for rate in interest_rates:
    df, total_interest, years = calculate_loan_fixed_monthly(
        loan_amount, 
        fixed_monthly_payment, 
        rate/100
    )
    
    df['Bank'] = bank_profiles[rate]["name"]
    all_data = pd.concat([all_data, df])
    max_years = max(max_years, years)
    
    avg_annual_repayment = df['Repayment'].mean()
    repayment_percentage = (avg_annual_repayment / loan_amount) * 100
    
    results.append({
        'Bank': bank_profiles[rate]["name"],
        'Interest Rate': f"{rate}%",
        'Monthly Payment': f"{fixed_monthly_payment:,} €".replace(",", "."),
        'Effective Repayment Rate': f"{repayment_percentage:.2f}%",
        'Total Interest Paid': f"{round(total_interest):,} €".replace(",", "."),
        'Loan Term': f"{years} years",
        'Final Payment': f"{df.iloc[-1]['Repayment']:,} €".replace(",", ".")
    })

# ====== Visualization ======
plt.style.use('default')
plt.rcParams.update({
    'figure.facecolor': 'white',
    'axes.grid': True,
    'grid.color': '#f0f0f0',
    'axes.facecolor': '#f9f9f9',
    'font.family': 'Arial',
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'legend.fontsize': 11
})

def create_plot(y_var, title, ylabel):
    fig, ax = plt.subplots(figsize=(14, 7))
    
    for rate in interest_rates:
        bank = bank_profiles[rate]
        df = all_data[all_data['Rate'] == rate/100]
        
        # Pad data for consistent x-axis
        x = np.arange(1, max_years + 1)
        y = np.zeros(max_years)
        y[:len(df)] = df[y_var]/1000
        
        ax.plot(x, y, 
                color=bank["color"], 
                linewidth=2.5,
                marker=bank["marker"],
                markersize=7,
                markevery=1,
                alpha=0.9,
                label=f"{bank['name']} ({rate}%)")
    
    ax.set_title(title, pad=20, fontweight='bold')
    ax.set_xlabel('Year', labelpad=12)
    ax.set_ylabel(ylabel, labelpad=12)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x):,}".replace(",", ".")))
    ax.set_axisbelow(True)
    ax.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Create plots
create_plot('Interest', 'Annual Interest Payments by Bank', 'Interest (in 1000 €)')
create_plot('Repayment', 'Annual Repayment Amounts by Bank', 'Repayment (in 1000 €)')

# ====== Results Table ======
results_df = pd.DataFrame(results).set_index('Bank')

# Format display
with pd.option_context('display.max_rows', None, 
                       'display.max_columns', None,
                       'display.width', None,
                       'display.colheader_justify', 'center'):
    print(f"\n🏦 Bank Loan Comparison (Fixed Monthly Payment: {fixed_monthly_payment:,} €)".replace(",", "."))
    display(results_df.style.set_properties(**{
        'text-align': 'center',
        'border': '1px solid #e0e0e0'
    }).format(precision=2))

# Additional analysis
print("\n🔍 Key Insights:")
for rate in interest_rates:
    bank = bank_profiles[rate]
    df = all_data[all_data['Rate'] == rate/100]
    total_interest = df['Interest'].sum()
    print(f"- {bank['name']}: {total_interest/loan_amount*100:.1f}% of loan amount paid as interest")


🏦 Bank Loan Comparison (Fixed Monthly Payment: 1.500 €)

🔍 Key Insights:
- Basisniveau: 50.3% of loan amount paid as interest
- Hypofin: 57.0% of loan amount paid as interest
- BBank: 70.9% of loan amount paid as interest
- Coba: 77.4% of loan amount paid as interest
- Zu teuer: 93.7% of loan amount paid as interest


When selecting the best bank offer for a loan, consider these **12 crucial factors** beyond just the interest rate:

---

### **1. Effective Interest Rate (APR)**
   - Includes all fees (processing, administration, etc.)
   - Example: A 2.5% rate with €5,000 fees may be worse than a 3.0% rate with no fees

### **2. Flexibility Features**
   - **Early repayment options** (penalty-free partial repayments)
   - **Payment holidays** (temporary payment suspensions)
   - **Rate conversion options** (fixed to variable or vice versa)

### **3. Loan Term Impact**
   - Shorter terms = less total interest but higher monthly payments
   - Use this formula to compare total cost:  
     ```
     Total Cost = (Monthly Payment × Term Months) + Fees
     ```

### **4. Fixed vs Variable Rates**
   | Fixed Rate | Variable Rate |
   |------------|---------------|
   | Predictable payments | Potential savings if rates drop |
   | Higher initial rate | Risk of rising payments |

### **5. Hidden Costs Checklist**
   - ❏ Valuation fees
   - ❏ Notary costs
   - ❏ Insurance bundling requirements
   - ❏ Late payment penalties

### **6. Bank Stability Ratings**
   Check the bank's:
   - Moody's/S&P ratings (A+ to D)
   - Non-performing loan ratio (<5% is good)
   - Recent profitability

### **7. Customer Service**
   - Average loan approval time (benchmark: <5 working days)
   - Digital tools quality (app functionality)
   - Branch availability for urgent issues

### **8. Insurance Requirements**
   - Some banks mandate:
     - Life insurance (0.1-0.3% of loan amount/year)
     - Home insurance (€200-€800/year)
   - Can add 15-30% to total loan cost

### **9. Collateral Terms**
   - Loan-to-value ratio (LTV) differences:
     | Bank Type | Typical Max LTV |
     |-----------|-----------------|
     | Traditional | 80% |
     | Online Banks | 90% |

### **10. Special Programs**
   - First-time buyer discounts
   - Green energy efficiency bonuses (up to 0.5% rate reduction)
   - Professional association partnerships

### **11. Prepayment Scenarios**
   Calculate break-even points:
   ```
   Break-even Year = Fees ÷ (Monthly Savings from Lower Rate)
   ```
   - Example: €3,000 fees / €50 monthly savings = 5 years

### **12. Regulatory Protections**
   - Deposit guarantee schemes coverage (up to €100k in EU)
   - Right of withdrawal period (typically 14 days)

---

### **Comparison Tool Recommendation**
Create a decision matrix like this:

| Criteria          | Weight | Bank A | Bank B | Bank C |
|-------------------|--------|--------|--------|--------|
| APR               | 30%    | 3.2%   | 3.5%   | 3.0%   |
| Flexibility       | 20%    | ⭐⭐⭐⭐ | ⭐⭐⭐   | ⭐⭐    |
| Fees              | 15%    | €1,200 | €800   | €2,000 |
| Customer Service  | 10%    | ⭐⭐    | ⭐⭐⭐⭐ | ⭐⭐⭐   |
| Insurance Costs   | 10%    | €600/yr| €850/yr| €400/yr|
| **Total Score**   |        | 82     | 76     | 78     |

---

### **Pro Tip**
Always request **binding offers in writing** from at least 3 banks, then:
1. Compare using the matrix above
2. Negotiate better terms using competing offers
3. Consult an independent mortgage advisor (cost: 0.5-1% of loan, but can save 5-15% long-term)

The "best" offer balances **short-term affordability** with **long-term flexibility** and **risk protection**.